# Building a Content-based Recommender System using Blue Brain Nexus

The goal of this notebook is to create a pipeline to train knowledge embedding model from some movie rating data and export it in Elasticsearch within Blue Brain Nexus. Once exported, you can test your recommendations by querying Elasticsearch and displaying the results.

![Movie Recommendation](https://raw.githubusercontent.com/BlueBrain/nexus-bbp-domains/docs/src/main/paradox/docs/bluebrainnexustutorialkcni/build-recommender-kgembeddings/assets/ml_datapipeline.png)


### _Prerequisites_

We will work with the small version of the MovieLens dataset containing a set of movies (movies.csv). An overview of this dataset can be found [here](https://bluebrainnexus.io/docs/tutorial/getting-started/dataset/index.html). The data is preloaded in the following [tutorialnexus/movies](https://sandbox.bluebrainnexus.io/web/tutorialnexus/movies) project that can be browsed. 


## Overview

You will work through the following steps

1. Set up the Nexus environment in Python 
2. Pull data from Nexus
3. Prepare the data and ensure they are in a good shape for embeddings
4. Train the recommendation model
5. Push the entity results to a ElasticSearch view in Nexus
6. Recommend similar items using 


### Uncomment the code below if you are running this notebook in Google Colab

In [0]:

"""
import os 

tutorial_base_dir = "/content/nexus-bbp-domains"
if os.path.exists(tutorial_base_dir):
  !rm -R $tutorial_base_dir

!git clone --single-branch --branch docs https://github.com/BlueBrain/nexus-bbp-domains.git
os.chdir("/".join([tutorial_base_dir,"src/main/paradox/docs/bluebrainnexustutorialkcni/notebooks"]))

print("The working directory is now:")
!pwd

"""

Cloning into 'nexus-bbp-domains'...
remote: Enumerating objects: 469, done.
remote: Counting objects: 100% (469/469), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 4221 (delta 197), reused 400 (delta 149), pack-reused 3752
Receiving objects: 100% (4221/4221), 14.07 MiB | 20.15 MiB/s, done.
Resolving deltas: 100% (1249/1249), done.
The working directory is now:
/content/nexus-bbp-domains/src/main/paradox/docs/bluebrainnexustutorialkcni/notebooks


## Step 1: Set up Nexus environment

In this step, you will set up a Nexus client with:
* An access token. Go to [Nexus Web](https://sandbox.bluebrainnexus.io/web) to login and get a token.
* An organisation and a project

You will also create a sparql client around your project to be able to query data.

In [0]:
import getpass

In [0]:
token = getpass.getpass()

··········


In [0]:
#install the Blue Brain Nexus python SDK
%%capture 
!pip install -U nexus-sdk

In [0]:
#Configuration for the Nexus deployment
nexus_deployment = "https://sandbox.bluebrainnexus.io/v1"

# set organization and project
org ="tutorialnexus"
project ="movies"

import nexussdk as nexus
nexus.config.set_environment(nexus_deployment)
nexus.config.set_token(token)

In [0]:
#Utils function
from pygments.lexers import JsonLexer
from pygments import highlight
from pygments.formatters import TerminalFormatter
from collections import OrderedDict
import json

def pretty_print(payload:dict):
    if type(payload) == OrderedDict:
        payload = json.loads(json.dumps(payload))
    print(highlight(json.dumps(payload, indent=4), JsonLexer(), TerminalFormatter()))

In [0]:
# Verify if the Nexus client is working by listing the organisations created in the sandbox

pretty_print(nexus.organizations.list())

### Create a sparql wrapper around your project's SparqlView

Every project in Blue Brain Nexus comes with a SparqlView enabling to navigate the data as a graph and to query it using the W3C SPARQL Language. The address of such SparqlView is https://nexus-sandbox.io/v1/views/tutorialnexus/PROJECTLABEL/graph/sparql for a project with the label $PROJECTLABEL. The address of a SparqlView is also called a SPARQL endpoint.

In [0]:
#Let install sparqlwrapper which a python wrapper around sparql endpoints
%%capture 
!pip install git+https://github.com/RDFLib/sparqlwrapper

In [0]:
# Utility functions to create sparql wrapper around a sparql endpoint

from SPARQLWrapper import SPARQLWrapper, JSON, POST, GET, POSTDIRECTLY, CSV
import requests
import pandas as pd
pd.set_option('display.max_colwidth', -1)

# Create a SPARQL client
def create_sparql_client(sparql_endpoint, http_query_method=POST, result_format= JSON, token=None):
    sparql_client = SPARQLWrapper(sparql_endpoint)
    #sparql_client.addCustomHttpHeader("Content-Type", "application/sparql-query")
    if token:
        sparql_client.addCustomHttpHeader("Authorization","Bearer {}".format(token))
    sparql_client.setMethod(http_query_method)
    sparql_client.setReturnFormat(result_format)
    if http_query_method == POST:
        sparql_client.setRequestMethod(POSTDIRECTLY)
    return sparql_client



# Convert SPARQL results into a Pandas data frame
def sparql2dataframe(json_sparql_results):
    cols = json_sparql_results['head']['vars']
    out = []
    for row in json_sparql_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

# Send a query using a sparql wrapper 
def query_sparql(query, sparql_client):
    sparql_client.setQuery(query)
    

    result_object = sparql_client.query()
    if sparql_client.returnFormat == JSON:
        return result_object._convertJSON()
    return result_object.convert()

In [0]:
# Let create a sparql client around the project sparql view
sparqlview_endpoint = nexus_deployment+"/views/"+org+"/"+project+"/graph/sparql"
sparqlview_wrapper = create_sparql_client(sparql_endpoint=sparqlview_endpoint, token=token,http_query_method= POST, result_format=JSON)


## Step 2: Pull and explore data from Nexus

Data that has been ingested into your Nexus project previously can now be queried. 

For building a classical recommendation system using matrix factorization, we will need a user-by-item matrix where nonzero elements of the matrix are ratings that a user has given an item. To do that, we will 

1.   Query all the rating data for building a U-I matrix
2.   Query the movie id data for the recomendation

SPARQL is an RDF query language which is able to retrieve and manipulate data stored in Resource Description Framework (RDF) format. Given that all the movielens data is put into the knowledge graph meaning they are inter-connected, it is straightforward to query the data using SPARQL.

In [0]:



entity_namespace = "https://sandbox.bluebrainnexus.io/v1/vocabs/tutorialnexus/movies/"
movie_type = entity_namespace+"Movie"

embedding_property = entity_namespace+"embedding"
total_result = 300000
page_size = 10000
offset = 0

count = 0
nexus_df=None
while ( count <= total_result ): 
    select_all_query = """
                        SELECT ?s ?p ?o
                        WHERE
                        {
                          ?s a <%movie_type%>.
                          ?s ?p ?o.
                          FILTER(?p != <%embedding_property%>).
                          FILTER (!isBlank(?p)).
                          FILTER (!isBlank(?o)).
                        }
                        OFFSET %offset%
                        LIMIT %page_size%
                        """
    select_all_query = select_all_query.replace("%offset%",str(offset)) \
                                       .replace("%page_size%",str(page_size)) \
                                       .replace("%movie_type%",str(movie_type)) \
                                       .replace("%embedding_property%",str(embedding_property))
    
    nexus_results = query_sparql(select_all_query,sparqlview_wrapper)

    result_df =sparql2dataframe(nexus_results)
    
    
    if len(result_df.index) == 0:
        break;
        
    if nexus_df is None:
        
        nexus_df = pd.DataFrame(result_df)
    else:
        
        nexus_df = pd.concat([nexus_df,result_df],ignore_index=True)
        
    count = count + page_size
    offset = offset+page_size


# Let explore the data

In [0]:
display(nexus_df.shape)
display(nexus_df.describe())
display(nexus_df.head())


(165614, 3)

,s,p,o
count,165614,165614,165614
unique,6590,17,54372
top,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/movies/_/Movie_172637,https://sandbox.bluebrainnexus.io/v1/vocabs/tutorialnexus/movies/imdbId,https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy
freq,68,9750,19475


,s,p,o
0,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/movies/_/Movie_7,https://bluebrain.github.io/nexus/vocabulary/constrainedBy,https://bluebrain.github.io/nexus/schemas/unconstrained.json
1,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/movies/_/Movie_7,https://bluebrain.github.io/nexus/vocabulary/createdAt,2019-11-21T16:22:49.875Z
2,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/movies/_/Movie_7,https://bluebrain.github.io/nexus/vocabulary/createdBy,https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy
3,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/movies/_/Movie_7,https://bluebrain.github.io/nexus/vocabulary/deprecated,false
4,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/movies/_/Movie_7,https://bluebrain.github.io/nexus/vocabulary/incoming,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/movies/_/https%3A%2F%2Fsandbox.bluebrainnexus.io%2Fv1%2Fresources%2Ftutorialnexus%2Fmovies%2F_%2FMovie_7/incoming


In [0]:
#nexus_df = pd.DataFrame()
nexus_df.to_csv("./movies_triples.csv",index=False)

## Step 3 Prepare the data

This step is about cleaing the data so that they can be in a 'good' shape for training embeddings

* Remove Nexus added metadata
* Remove namespaces
* Remove numerical properties such as tmdbId

In [0]:
%%capture 
!pip install validators

In [0]:
import validators

is_nexus_vocab_prop = nexus_df['p'].str.startswith("https://bluebrain.github.io/nexus/vocabulary/")
is_nexus_vocab_sub = nexus_df['s'].str.startswith("https://bluebrain.github.io/nexus/vocabulary/")

nexus_df_filtered = nexus_df[is_nexus_vocab_prop==False]
nexus_df_filtered = nexus_df_filtered[is_nexus_vocab_sub==False]


nexus_df_filtered = nexus_df_filtered[pd.notnull(nexus_df_filtered['o'])]
nexus_df_filtered.fillna("")

def keep_fragment(x):
    
    if validators.url(x):
        if "#" not in x:
            x_parts =str(x).split("/")
            if x_parts[-1] != "embedding":
                return x_parts[-1]
        else:
            x_parts =str(x).split("#")
            return x_parts[-1]
    else:
        return x

nexus_df_filtered['s'] = nexus_df_filtered['s'].apply(lambda x: keep_fragment(x))
nexus_df_filtered['p'] = nexus_df_filtered['p'].apply(lambda x: keep_fragment(x))
nexus_df_filtered['o'] = nexus_df_filtered['o'].apply(lambda x: keep_fragment(x))

nexus_df_filtered['o'].astype(str)
nexus_df_filtered=nexus_df_filtered[nexus_df_filtered['p'] !="tmdbId"]
nexus_df_filtered=nexus_df_filtered[nexus_df_filtered['p']!="imdbId"]
nexus_df_filtered=nexus_df_filtered[nexus_df_filtered['p'] !="movieId"]




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [0]:
display(nexus_df_filtered.shape)
display(nexus_df_filtered.describe())
display(nexus_df_filtered.head())

(29233, 3)

,s,p,o
count,29233,29233,29233
unique,6544,3,7290
top,Movie_171891,genres,Movie
freq,12,9749,9743


,s,p,o
11,Movie_7,genres,Comedy|Romance
14,Movie_7,title,Sabrina (1995)
16,Movie_7,type,Movie
28,Movie_2,genres,Adventure|Children|Fantasy
31,Movie_2,title,Jumanji (1995)


In [0]:
nexus_df_filtered.to_csv("./movies_triples_prepared.csv",index=False, header=False)

## Step 4: Train a recommmender model

We will use te Ampligraph Library implementing many [knowledge graph embeddings](https://docs.ampligraph.org/en/1.1.0/ampligraph.latent_features.html). Click to have more details about those models.



In [0]:
%%capture 
!pip install ampligraph --ignore-installed

In [0]:
import ampligraph
!python --version

Python 3.6.8


In [0]:
import numpy as np
from ampligraph.datasets import load_wn18, load_wn18rr, load_fb15k_237
from ampligraph.latent_features import ComplEx, TransE,HolE
from ampligraph.evaluation import evaluate_performance, mrr_score, hits_at_n_score
from ampligraph.utils import save_model
from ampligraph.evaluation import train_test_split_no_unseen

Build train and test dataset

In [0]:
# Since AmpliGraph only support loading from files, we will load the prepared dataset we saved before.
data = ampligraph.datasets.load_from_csv(directory_path=".",file_name="movies_triples_prepared.csv", sep=",")
num_test = int(len(data) * (20 / 100))
train, test = train_test_split_no_unseen(data, test_size=num_test, seed=0, allow_duplication=False) 
print('Train set size: ', train.shape)
print('Test set size: ', test.shape)

Train set size:  (15647, 3)
Test set size:  (3911, 3)


Lets go through the parameters to understand what's going on:

- **`k`** : the dimensionality of the embedding space
- **`eta`** ($\eta$) : the number of negative, or false triples that must be generated at training runtime for each positive, or true triple
- **`batches_count`** : the number of batches in which the training set is split during the training loop. If you are having into low memory issues than settings this to a higher number may help.
- **`epochs`** : the number of epochs to train the model for.
- **`optimizer`** : the Adam optimizer, with a learning rate of 1e-3 set via the *optimizer_params* kwarg.
- **`loss`** : pairwise loss, with a margin of 0.5 set via the *loss_params* kwarg.
- **`regularizer`** : $L_p$ regularization with $p=2$, i.e. l2 regularization. $\lambda$ = 1e-5, set via the *regularizer_params* kwarg. 

Now we can instantiate the model:


In [0]:
model = ComplEx(batches_count=10, seed=0, epochs=20, k=150, eta=10,
                    # Use adam optimizer with learning rate 1e-3
                    optimizer='adam', optimizer_params={'lr':1e-3},
                    # Use pairwise loss with margin 0.5
                    loss='pairwise', loss_params={'margin':0.5},
                    # Use L2 regularizer with regularizer weight 1e-5
                    regularizer='LP', regularizer_params={'p':2, 'lambda':1e-5}, 
                    # Enable stdout messages (set to false if you don't want to display)
                    verbose=True)

In [0]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(train, early_stopping = False)

Average Loss:   0.024999: 100%|██████████| 20/20 [00:38<00:00,  1.88s/epoch]


In [0]:
from ampligraph.utils import create_tensorboard_visualizations, restore_model
model_name_path = './movies_embeddings_model.pkl'
restored_model = restore_model(model_name_path= model_name_path)
output_path = './Graph'


In [0]:
embeddings = restored_model.get_embeddings(['Movie_5','Movie_9'], embedding_type='entity')
print(embeddings.shape)

(2, 300)


In [0]:
from numpy import dot
from numpy.linalg import norm
import json
sim = dot(embeddings[0], embeddings[1])/(norm(embeddings[0])*norm(embeddings[1]))
print(sim)

0.042629305


## Step 5 Push the embeddings to an ElasticSearch view in Nexus

* Update every item in Nexus by adding their corresponding embedding
* Create an ElasticSearch view to index and serve the items along with their embeddings

In [0]:

project_with_embedding = "myverycoolproj" #create a new project
response = nexus.projects.create(org,project_with_embedding )
pretty_print(response)

In [0]:

def update_resource(nexus,identifier, org, project_source, project_target, embedding):
    try:
        resource = nexus.resources.fetch(org_label=org,project_label=project_source,resource_id=identifier)
        resource = json.loads(json.dumps(resource))
        resource_current_revision = resource["_rev"]
        
        

        project_target_ns="https://sandbox.bluebrainnexus.io/v1/resources/"+org+"/"+project_target+"/_/"

        
        resource["@context"][0]["embedding"] = {
            "@container":"@list"
        }
        resource["@context"][0]["@base"] = project_target_ns
        resource["@context"][0]["@vocab"] = 'https://sandbox.bluebrainnexus.io/v1/vocabs/tutorialnexus/movies/'
        
      
        resource['embedding'] = embedding.tolist()
        
        
       
        resource["@id"] = project_target_ns+resource["@id"]
        resource["_self"] = resource["@id"]
        
        response = nexus.resources.create(org,project_target,data=resource,resource_id=resource["@id"])
        return response
    except nexus.HTTPError as e:
      print(e)
      return str(e)

def update_resource_old(nexus,identifier, org, project_source, project_target, embedding):
    try:
        resource = nexus.resources.fetch(org_label=org,project_label=project_source,resource_id=identifier)
        resource = json.loads(json.dumps(resource))
        resource_current_revision = resource["_rev"]
        
        resource["@context"][0]["embedding"] = {
            "@container":"@list"
        }
       # print(embedding)
        #print(type(embedding))
        resource['embedding'] = embedding.tolist()
        #pretty_print(resource)
        response = nexus.resources.update(resource=resource, rev=resource_current_revision)
        return response
    except nexus.HTTPError as e:
        return str(e)
    
#Add embeddings to the data

def update_with_embedding(number_of_movies,project_source, project_target):
    
    ns="https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/"+project_source+"/_/"

    
    items = nexus_df_filtered["s"].unique()
    i = 0
    items = items[:number_of_movies]
    for item in items:
        try:
            item_embedding = restored_model.get_embeddings([item], embedding_type='entity')
            
            response = update_resource(nexus,ns+item,org,project_source,project_target,item_embedding[0])
            i= i+1
            print("Created: "+str(item))
            
        except nexus.HTTPError as e:
            print (e.response.json())
    print(i)

In [0]:
#Load 100 movies in the newly created project
update_with_embedding(100,project,project_with_embedding)

### Create an ElasticSearchView

The goal here is to illustrate hwo to use the Nexus SDK to create an Elasticsearch view. The full documentation can be found at: https://bluebrainnexus.io/docs/api/current/kg/kg-views-api.html#create-an-elasticsearchview-using-post.

```
{
  "@id": "{someid}",
  "@type": [ "View", "ElasticSearchView"],
  "resourceSchemas": [ "{resourceSchema}", ...],
  "resourceTypes": [ "{resourceType}", ...],
  "resourceTag": "{tag}",
  "includeMetadata": {includeMetadata},
  "includeDeprecated": {includeDeprecated},
  "mapping": _elasticsearch mapping_
}

````

An ElasticSearchView is a way to tell Nexus:

* Which resources to index in the view:

 * resources that conform to a given schema: set resourceSchemas to the targeted schemas

 * resources that are of a given type: set resourceTypes to the targeted types

 * resources that are tagged: set resourceTag to the targeted tag value.

* Which mapping to use when indexing the selected resources:

 * set mappingto be used: More info about Elasticsearch mapping.

In [0]:
type_to_index = ["https://sandbox.bluebrainnexus.io/v1/vocabs/tutorialnexus/movies/Movie"]
view_id="https://mymovieview.org"
view_data = {
    "@type": [
        "ElasticSearchView"
    ],
    "includeMetadata": True,
    "includeDeprecated": False,
    "resourceTypes":type_to_index,
    "mapping": {
        "properties": {
            "@id": {
                "type": "keyword"
            },
            "@type": {
                "type": "keyword"
            },
            "embedding": {
                "type":"dense_vector",
                "dims":300
            }
        }
    },
    "sourceAsText": False
}

try:
    response = nexus.views.create_(org_label=org, project_label=project_with_embedding,payload=view_data,view_id=view_id)
    pretty_print(response)
except nexus.HTTPError as ne:
    pretty_print(ne.response.json())


Go to [Nexus web](https://sandbox.bluebrainnexus.io/web/tutorialnexus). Select your project and view the indexing progress for the view https://mymovieview.org

## Step 6 Get similar item Using ElasticSearch

From a movie identifier get similar movies. The following method will fetch from Nexus the corresponding entity, extract its embedding and use it to build an ElasticSearch query submitted to an ElasticSearch view.

In [0]:
def get_similar_movies(item_id, q="*", number_of_results=10, view_id="demo"):
    """
    Given a movie id, execute the recommendation function score query to find similar movies, ranked by cosine similarity
    """
    # Get the item from Nexus and retrieve its embedding
    
    item_source = nexus.resources.fetch(org_label=org,project_label=project_with_embedding,resource_id=item_id)
    item_source = json.loads(json.dumps(item_source))
    
    # extract the embedding
    item_embedding = item_source['embedding']
    query = {
          "size": number_of_results,
                "_source": {
                "exclude": [ "embedding" ]
           },
          "query": {
            "script_score": {
              "query": {
                    "exists": {
                    "field": "embedding"
                    }
              },
              "script": {
                "source": "cosineSimilarity(params.queryVector, doc['embedding'])+1.0",
                "params": {
                  "queryVector": item_embedding
                }
              }
            }
          }
        }

    response = nexus.views.query_es(org_label=org, project_label=project_with_embedding,query=query,view_id=view_id)
    response_json = json.loads(json.dumps(response))
    
    number_of_results = response["hits"]['total']['value']
    similar_movies = [(hit["_source"]["title"],
                       hit["_source"]["genres"],
                       hit["_score"],
                       hit["_source"]["movieId"])
                       for hit in response["hits"]['hits']]
    return number_of_results, similar_movies
    


In [0]:
#query similar data
view_id="https://mymovieview.org"
number_of_results = 10
movieId = "Movie_349"
result = nexus_df_filtered.loc[nexus_df_filtered['s'] == movieId]
print("Looking for movies similar to %s" %(movieId))
display(result)
project_target_ns="https://sandbox.bluebrainnexus.io/v1/resources/"+org+"/"+project_with_embedding+"/_/"
number_of_similar_movies, similar_movies = get_similar_movies(item_id= project_target_ns+movieId,number_of_results=number_of_results,view_id=view_id)
print("\n Found %s similar movies. \nShowing %s movies" %(str(number_of_similar_movies),str(number_of_results)))

similar_movies_df = pd.DataFrame(similar_movies, columns =['Title', 'Genres', 'Score', "Id"]) 
similar_movies_df.head(10)    


Looking for movies similar to Movie_349


,s,p,o
1062,Movie_349,genres,Action|Crime|Drama|Thriller
1065,Movie_349,title,Clear and Present Danger (1994)
1067,Movie_349,type,Movie



 Found 92 similar movies. 
Showing 10 movies


,Title,Genres,Score,Id
0,Clear and Present Danger (1994),Action|Crime|Drama|Thriller,2.000000,349
1,Fresh (1994),Crime|Drama|Thriller,1.303964,456
2,Eat Drink Man Woman (Yin shi nan nu) (1994),Comedy|Drama|Romance,1.303016,232
3,Interview with the Vampire: The Vampire Chronicles (1994),Drama|Horror,1.275722,253
4,Amateur (1994),Crime|Drama|Thriller,1.264097,149
5,Forget Paris (1995),Comedy|Romance,1.263060,237
6,"Young Poisoner's Handbook, The (1995)",Crime|Drama,1.258947,117
7,Casino (1995),Crime|Drama,1.249238,16
8,"American President, The (1995)",Comedy|Drama|Romance,1.243004,11
9,Waiting to Exhale (1995),Comedy|Drama|Romance,1.233743,4
